In [191]:
import pandas as pd
import xgboost as xgb
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, average_precision_score, roc_auc_score, classification_report
import osmnx
import utm
import numpy as np
import random

In [192]:
!pip install osmnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [193]:
!pip install utm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [194]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [195]:
df = pd.read_csv("/content/drive/MyDrive/final_ev.csv")
df = df.drop(['Unnamed: 0'], axis=1)

In [196]:
df.head()

,latitude,longitude,college,food_court,hospital,parking,restaurant,university,charging_station
0,51.510359,-0.187704,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,51.403444,-0.255276,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,51.465290,-0.058371,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,51.438312,-0.156274,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,51.358167,-0.216632,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [197]:
X = df.drop(['charging_station'], axis=1)
y=df['charging_station']

In [198]:
X = X.to_numpy()
y = y.to_numpy()

In [199]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0, test_size=0.2)

In [200]:
# using the XGBClassifier
cls = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
cls.fit(X_train, y_train)

XGBClassifier(random_state=42)

In [201]:
labels = cls.predict(X_train)
print(classification_report(y_train, labels))
print('Confusion Matrix')
print(confusion_matrix(y_train, labels))

              precision    recall  f1-score   support

         0.0       0.86      1.00      0.92      3492
         1.0       1.00      0.81      0.89      2960

    accuracy                           0.91      6452
   macro avg       0.93      0.90      0.91      6452
weighted avg       0.92      0.91      0.91      6452

Confusion Matrix
[[3490    2]
 [ 565 2395]]


In [202]:
labels = cls.predict(X_test)
print(classification_report(y_test, labels))
print('Confusion Matrix')
print(confusion_matrix(y_test, labels))

              precision    recall  f1-score   support

         0.0       0.88      1.00      0.93       914
         1.0       1.00      0.82      0.90       700

    accuracy                           0.92      1614
   macro avg       0.94      0.91      0.92      1614
weighted avg       0.93      0.92      0.92      1614

Confusion Matrix
[[913   1]
 [129 571]]


In [203]:
pickle.dump(cls, open("/content/drive/MyDrive/optimal_pred_model.dat", "wb"))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ResourceWarning: unclosed file <_io.BufferedWriter name='/content/drive/MyDrive/optimal_pred_model.dat'>
  """Entry point for launching an IPython kernel.


In [224]:
# taking one gps coordinate from the user and finding 10 random locations around it to test if they are optimal or not
latitude = 51.515650

longitude = -0.199502

n = 10

this_map = folium.Map(prefer_canvas=True)

def plotDotCS(latitude,longitude, this_map):
  
  folium.Marker(
  location=[latitude, longitude],
  ).add_to(this_map)

coordinate_list = []


for i in range(n):
    if i<i/2:
      flt = float(random.randint(0,50))
      dec_lat = random.random()/100
      dec_lon = random.random()/100
      plotDotCS(latitude+dec_lon, longitude+dec_lat, this_map)
      coordinate_list.append((latitude+dec_lon, longitude+dec_lat))
    else:

      flt = float(random.randint(0,50))
      dec_lat = random.random()/100
      dec_lon = random.random()/100
      plotDotCS(latitude-dec_lon, longitude-dec_lat, this_map)
      coordinate_list.append((latitude-dec_lon, longitude-dec_lat))
    



this_map.fit_bounds(this_map.get_bounds())


this_map

In [225]:

new_model = pickle.load(open("/content/drive/MyDrive/optimal_pred_model.dat", "rb"))

def predict_optimal_place(lat,lon, new_model):


  # converting lat, lon from WGS4 to UT,
  easting, northing, _, _ = utm.from_latlon(lat, lon)

  # adding 250 metres buffer to the user input coordinates to give surrounding amenity data
  easting = easting + 200
  northing = northing + 200

  # converting back to lat, lon

  lat1, lon1 = utm.to_latlon(easting, northing, 30, 'U')
  
  # amenities that we are considering
  tags = {'amenity': ['food_court', 'restaurant', 'college', 'university', 'parking', 'hospital']}
  
  # fetching suitable amenity location through OpenStreetMap
  x = osmnx.geometries.geometries_from_bbox(north=lat1, south=lat, east=lon1, west=lon, tags=tags)

  emp = []
  if (len(x.columns.values.tolist())<=1):
    return 0, lat, lon, emp

  x = x[['amenity']]
  x = pd.get_dummies(x['amenity'])

  possible_amenities = {
      'food_court' : 0,
      'restaurant' : 0,
      'college' : 0,
      'university': 0,
      'parking': 0, 
      'hospital': 0
 }

  captured_variables = x.columns.values.tolist()

  for amenity in captured_variables:
    if x[amenity].sum()>0:
      possible_amenities.update({amenity:1})
  
  ip_features = np.array([lat, lon, possible_amenities['college'], possible_amenities['food_court'], possible_amenities['hospital'], possible_amenities['parking'], possible_amenities['restaurant'], possible_amenities['university']])
  
  pred = new_model.predict(ip_features.reshape(-1,1).T)

  del new_model

  return pred[0], lat, lon, captured_variables

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ResourceWarning: unclosed file <_io.BufferedReader name='/content/drive/MyDrive/optimal_pred_model.dat'>
  """Entry point for launching an IPython kernel.


In [226]:
this_map = folium.Map(prefer_canvas=True)

# function for 
def plotDotCS2(latitude,longitude, st, pred, this_map):
  
  if pred==1:
    color = 'green'
  else:
    color = 'red'

  icone1 = folium.Icon(icon="asterisk", icon_color="white", color=color)

  folium.Marker(
  location=[lat, lon],
  popup=st,
  icon=icone1,
  ).add_to(this_map)

# going through all the location and determining whether they are optimaal
for coordinate in coordinate_list:
  ilat, ilong = coordinate
  pred, lat, lon, captured_variables = predict_optimal_place(ilat, ilong, new_model)

  st = "Nearby: " 
  i = 0
  for amenity in captured_variables:
    i = i + 1
    if i==len(captured_variables):
      st = st + amenity
    else:
      st = st + amenity + ', '

  plotDotCS2(lat, lon, st, pred, this_map)

# plotting location on map

this_map.fit_bounds(this_map.get_bounds())

this_map.save('/content/drive/MyDrive/res.html')


## Green - optimal
## Red - not optimal

In [227]:
this_map

In [215]:
# this_map.save('/content/drive/MyDrive/res.html')

In [228]:
icone1 = folium.Icon(icon="asterisk", icon_color="white", color="blue")

folium.Marker(
location=[lat, lon],
popup=st,
icon=icone1,
).add_to(this_map)

this_map.fit_bounds(this_map.get_bounds())

this_map
